# Étape 3 - Chargement, pré-traitement

In [4]:
import com.google.gson.Gson

case class Car(
  rownames: Int,
  mpg: Double,
  cylinders: Int,
  displacement: Double,
  horsepower: Double,
  weight: Double,
  acceleration: Double,
  year: Int,
  origin: Int,
  name: String
)


val carDF = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv("./data-car.csv")
  .as[Car]


carDF.show()

+--------+----+---------+------------+----------+------+------------+----+------+--------------------+
|rownames| mpg|cylinders|displacement|horsepower|weight|acceleration|year|origin|                name|
+--------+----+---------+------------+----------+------+------------+----+------+--------------------+
|       1|18.0|        8|       307.0|       130|  3504|        12.0|  70|     1|chevrolet chevell...|
|       2|15.0|        8|       350.0|       165|  3693|        11.5|  70|     1|   buick skylark 320|
|       3|18.0|        8|       318.0|       150|  3436|        11.0|  70|     1|  plymouth satellite|
|       4|16.0|        8|       304.0|       150|  3433|        12.0|  70|     1|       amc rebel sst|
|       5|17.0|        8|       302.0|       140|  3449|        10.5|  70|     1|         ford torino|
|       6|15.0|        8|       429.0|       198|  4341|        10.0|  70|     1|    ford galaxie 500|
|       7|14.0|        8|       454.0|       220|  4354|         9.0|  70

defined class Car
carDF = [rownames: int, mpg: double ... 8 more fields]


[rownames: int, mpg: double ... 8 more fields]